<a href="https://colab.research.google.com/github/etuckerman/SOCOTEC/blob/main/SOCOTEC_GPU_elliot_tuckerman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Dataset Creation: Mapping Natural Language Prompts to Function Calls
dataset = [
    {"input": "Calculate the sum of 7 and 3.", "output": "add(7, 3)"},
    {"input": "What's the square of 5?", "output": "square(5)"},
    {"input": "Tell me the cube of 2.", "output": "cube(2)"},
    {"input": "Add 12 to 8.", "output": "add(12, 8)"},
    {"input": "What is 4 squared?", "output": "square(4)"},
    {"input": "Compute the cube of 4.", "output": "cube(4)"},
    {"input": "Say hello to John.", "output": "greet('John')"},
    {"input": "What's 10 + 20?", "output": "add(10, 20)"},
    {"input": "Find the square of 3.", "output": "square(3)"},
    {"input": "Calculate the cube of 5.", "output": "cube(5)"},
    {"input": "Greet Alice for me.", "output": "greet('Alice')"},
    {"input": "What's 2 + 3?", "output": "add(2, 3)"},
    {"input": "Calculate 6 squared.", "output": "square(6)"},
    {"input": "Give me the cube value of 8.", "output": "cube(8)"},
    {"input": "Say hi to Bob.", "output": "greet('Bob')"},
    {"input": "Add 15 and 9.", "output": "add(15, 9)"},
    {"input": "What is 7 cubed?", "output": "cube(7)"},
    {"input": "Calculate 4 cubed.", "output": "cube(4)"},
    {"input": "Square the number 6.", "output": "square(6)"},
    {"input": "Say hello to Sarah.", "output": "greet('Sarah')"},
    {"input": "What is 5 + 5?", "output": "add(5, 5)"},
    {"input": "What is 3 squared?", "output": "square(3)"},
    {"input": "Tell me the cube of 10.", "output": "cube(10)"}
]


In [11]:
# Function Library
def add(a, b):
    """Return the sum of two numbers."""
    return a + b

def square(a):
    """Return the square of a number."""
    return a ** 2

def cube(a):
    """Return the cube of a number."""
    return a ** 3

def greet(name):
    """Return a greeting string."""
    return f"Hello, {name}!"


In [12]:
!pip install transformers accelerate


In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Model name as specified
model_name = "Qwen/Qwen2.5-7B-Instruct"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

print("Model and tokenizer successfully loaded!")


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1390: UserWarning: Current model requires 7424 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Model and tokenizer successfully loaded!


In [27]:
def generate_response(prompt, max_tokens=100, temperature=0.7):
    """
    Generate LLM response with an adjusted max_token limit.
    """
    # Inject constraints into the prompt for function-like behavior
    system_prompt = """
    You are a helpful assistant. You will always respond by directly referencing function calls.
    Use the following functions: add(a, b), square(a), cube(a), greet(name).
    Do not explain; simply return the response using function calls only.
    Examples:
    - Input: "Calculate 7 + 3"
      Output: add(7, 3)
    - Input: "What's the square of 5?"
      Output: square(5)
    - Input: "Greet Alice"
      Output: greet('Alice')
    """

    # Combine system prompt with user input
    full_prompt = f"{system_prompt}\nUser: {prompt}\nResponse:"

    # Tokenize and send input to model with adjusted limits
    input_tokens = tokenizer(full_prompt, return_tensors="pt", truncation=True, max_length=512)

    # Generate response
    output_tokens = model.generate(
        **input_tokens,
        max_new_tokens=max_tokens,  # Dynamically allow more tokens for response
        temperature=temperature,
        pad_token_id=tokenizer.pad_token_id,
    )

    # Decode the response tokens
    response = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    return response.strip()


In [28]:
# Test the model
response = generate_response("What's the square of 5?")
print("Response from model:", response)


Response from model: You are a helpful assistant. You will always respond by directly referencing function calls.
    Use the following functions: add(a, b), square(a), cube(a), greet(name).
    Do not explain; simply return the response using function calls only.
    Examples:
    - Input: "Calculate 7 + 3"
      Output: add(7, 3)
    - Input: "What's the square of 5?"
      Output: square(5)
    - Input: "Greet Alice"
      Output: greet('Alice')
    
User: What's the square of 5?
Response: square(5)


In [29]:
def add(a, b):
    return a + b

def square(a):
    return a ** 2

def cube(a):
    return a ** 3

def greet(name):
    return f"Hello, {name}!"


In [30]:
import re

def parse_and_execute(response):
    """
    Parses the response from the LLM and executes the corresponding function safely.
    :param response: The response string from the LLM.
    :return: Result of the parsed function execution.
    """
    try:
        if match := re.search(r"add\((\d+),\s*(\d+)\)", response):
            a, b = int(match.group(1)), int(match.group(2))
            return add(a, b)

        if match := re.search(r"square\((\d+)\)", response):
            num = int(match.group(1))
            return square(num)

        if match := re.search(r"cube\((\d+)\)", response):
            num = int(match.group(1))
            return cube(num)

        if match := re.search(r"greet\(['\"]([^'\"]+)['\"]\)", response):
            name = match.group(1)
            return greet(name)

        return "Function not recognized or invalid input"

    except Exception as e:
        return f"Error processing response: {e}"


In [31]:
# Complete interaction pipeline
user_input = "What's the square of 4?"

# Generate response from model
response = generate_response(user_input)

# Parse and execute the response
result = parse_and_execute(response)

print("Response from model:", response)
print("Executed result:", result)


Response from model: You are a helpful assistant. You will always respond by directly referencing function calls.
    Use the following functions: add(a, b), square(a), cube(a), greet(name).
    Do not explain; simply return the response using function calls only.
    Examples:
    - Input: "Calculate 7 + 3"
      Output: add(7, 3)
    - Input: "What's the square of 5?"
      Output: square(5)
    - Input: "Greet Alice"
      Output: greet('Alice')
    
User: What's the square of 4?
Response: square(4)
Executed result: 10


In [32]:
test_prompts = [
    "Calculate the sum of 7 and 3.",
    "Tell me the cube of 2.",
    "What's the square of 5?",
    "Greet Alice",
]

for prompt in test_prompts:
    response = generate_response(prompt)
    result = parse_and_execute(response)
    print(f"Prompt: {prompt}")
    print(f"Model Response: {response}")
    print(f"Execution Result: {result}")
    print("-" * 50)


Prompt: Calculate the sum of 7 and 3.
Model Response: You are a helpful assistant. You will always respond by directly referencing function calls.
    Use the following functions: add(a, b), square(a), cube(a), greet(name).
    Do not explain; simply return the response using function calls only.
    Examples:
    - Input: "Calculate 7 + 3"
      Output: add(7, 3)
    - Input: "What's the square of 5?"
      Output: square(5)
    - Input: "Greet Alice"
      Output: greet('Alice')
    
User: Calculate the sum of 7 and 3.
Response: add(7, 3)
Execution Result: 10
--------------------------------------------------
Prompt: Tell me the cube of 2.
Model Response: You are a helpful assistant. You will always respond by directly referencing function calls.
    Use the following functions: add(a, b), square(a), cube(a), greet(name).
    Do not explain; simply return the response using function calls only.
    Examples:
    - Input: "Calculate 7 + 3"
      Output: add(7, 3)
    - Input: "What's 